In [26]:
import sqlite3
import pandas as pd
import numpy as np
import hashlib
from pandasql import sqldf
from geopy.distance import great_circle

Part 1. Establishing Database

In [27]:
conn = sqlite3.connect("cs.db")

In [28]:
cur = conn.cursor()

In [29]:
#Tables on db
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",conn)

,name
0,fields
1,geo
2,inst
3,user


In [50]:
inst = pd.read_sql("SELECT * FROM inst",conn)

In [31]:
#conn.close()

Part 2. User Inputs

Preferences:
Input provided: Importance from scale 1 to 10
Output created: Allocation Percentage

In [32]:
#a, b, c represents importance user provides for each preference
a=4
b=5
c=6
raw_weights = np.array([a, b, c])
sum_weights = np.sum(raw_weights)
perc_weights = raw_weights/sum_weights

Part 2a. User ID Creation
Input provided: email
Output created: 1) user_id 2) inserts user_id and email into "user" table
Method: function (user_id_generator)

In [33]:
#Scenario 1 Testing
email = 'mkjones@gatech.edu'
user_id = int(hashlib.sha256(email.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID

In [34]:
user_id

9987952

In [35]:
#Scenario #2
#trying different email
email2 = 'michaelk.Jones@outlook.com'
user_id3 = int(hashlib.sha256(email2.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id3

80449489

In [36]:
#Scenario 3
#trying same email as scenario 2 but with all lowercase
email3 = 'michaelk.jones@outlook.com'
user_id4 = int(hashlib.sha256(email3.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id4

87012218

In [37]:
#Scenario 4
#same email as scenario 2 but added lower to have negate capitalization effects
email2 = 'michaelk.Jones@outlook.com'
user_id5 = int(hashlib.sha256(email2.lower().encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id5

87012218

In [38]:
def user_id_generator(email):
    id = int(hashlib.sha256(email.lower().encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
    return id

In [39]:
#test case of function
user_id_generator(email = 'test123@outlook.com')

83932234

In [40]:
email = 'test123@outlook.com'
user_id = user_id_generator(email = 'test123@outlook.com')

if pd.read_sql(f"SELECT * FROM user WHERE user_id ={user_id}",conn).empty:
    cur.execute(f"INSERT INTO user values('{email}',{user_id})")
else:
    print("Email exists")

In [42]:
pd.read_sql(f"SELECT * FROM user",conn)

,email,user_id
0,mkjones@gatech.edu,9987952
1,mykek.jones@gmail.com,95829733
2,michaelk.jones@outlook.com,87012218
3,test1@GMAIL.COM,65768313
4,test1@gmail.com,45591513


Part 2.b: Zipcode
user provided: zipcode
output created: dataframe of distance in miles per institution

In [43]:
geo = pd.read_sql("SELECT * FROM geo",conn)

In [44]:
geo

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,TRUE,None,17126,102.6,72001,Adjuntas,"{""72001"": 98.73, ""72141"": 1.27}",Adjuntas|Utuado,72001|72141,FALSE,FALSE,America/Puerto_Rico
1,00602,18.36075,-67.17541,Aguada,PR,Puerto Rico,TRUE,None,37895,482.5,72003,Aguada,"{""72003"": 100}",Aguada,72003,FALSE,FALSE,America/Puerto_Rico
2,00603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,TRUE,None,49136,552.4,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,FALSE,FALSE,America/Puerto_Rico
3,00606,18.16585,-66.93716,Maricao,PR,Puerto Rico,TRUE,None,5751,50.1,72093,Maricao,"{""72093"": 82.26, ""72153"": 11.68, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,FALSE,FALSE,America/Puerto_Rico
4,00610,18.2911,-67.12243,Anasco,PR,Puerto Rico,TRUE,None,26153,272.1,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.81, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,FALSE,FALSE,America/Puerto_Rico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33795,00614,18.4744398,-66.7197651,Arecibo,PR,,,,,,,,,,,,,
33796,00726,18.2344053,-66.048442,Caguas,PR,,,,,,,,,,,,,
33797,00928,18.4744398,-66.7197651,San Juan,PR,,,,,,,,,,,,,
33798,47903,40.4278046,-86.9111405,Lafayette,IN,,,,,,,,,,,,,


In [45]:
input_zip = "30315"

In [46]:
lat =float(list(geo[geo['zip']==input_zip]['lat'])[0])
lon =float(list(geo[geo['zip']==input_zip]['lng'])[0])
print(f"lat, lon: ({lat}, {lon})")

lat, lon: (33.70312, -84.38272)


In [51]:
inst[["UNITID","LATITUDE","LONGITUDE"]]

,UNITID,LATITUDE,LONGITUDE
0,100654,34.783368,-86.568502
1,100663,33.505697,-86.799345
2,100690,32.362609,-86.17401
3,100706,34.724557,-86.640449
4,100724,32.364317,-86.295677
...,...,...,...
6676,49576719,41.31559,-76.02235
6677,49576720,39.92142,-76.71277
6678,49576721,40.0466,-75.5302
6679,49576722,40.19437,-76.72659


In [57]:
## Function to use to find the distance of each university in miles
def inst_dist (lat,lon):
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    column_name = f"{user_id}_DISTANCE_MI"  # Adding user_id to the column title
    return inst.loc[:, ['UNITID']].assign(**{column_name: idx})

In [59]:
def inst_dist (lat,lon):
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    column_name = f"{user_id}_DISTANCE_MI"  # Adding user_id to the column title
    return inst.loc[:, ['UNITID']].assign(dist_miles = idx)

In [61]:
#Distances of each university to the input zip
dist_temp_table = inst_dist(lat,lon)
type(dist_temp_table)
dist_temp_table
inst = pd.merge(inst, dist_temp_table, on='UNITID')

Normalizing Features

creating features dict

In [66]:
def features_dict(feature1, feature2, feature3):
    featuresDict = {}
    featuresDict['feature1']= feature1
    featuresDict['feature2']= feature2
    featuresDict['feature3']= feature3
    return featuresDict

In [67]:
features_dict('OMENRUP_PELL_NFT_POOLED_SUPP', 'OMENRYP_PELL_FT_POOLED_SUPP', 'OMENRAP_PELL_FT_POOLED_SUPP')

{'feature1': 'OMENRUP_PELL_NFT_POOLED_SUPP',
 'feature2': 'OMENRYP_PELL_FT_POOLED_SUPP',
 'feature3': 'OMENRAP_PELL_FT_POOLED_SUPP'}

grabbing feature columns

In [ ]:
featuresQuery = """
SELECT
UNITID, {}, {}, {}
FROM inst
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'])

In [ ]:
featuresdf2 = pd.read_sql(featuresQuery,conn)

In [ ]:
featuresDF = sqldf(featuresQuery,locals())
featuresDF.shape[0]
featuresDF

In [ ]:
featuresdf2
featuresdf2.shape[0]

In [ ]:
test = pd.merge(featuresDF, dist_temp_table, on='UNITID')
test

Normalizing each feature

part 1: find each feature's length

In [ ]:
maxQuery = """
SELECT max({0}) x1FeatureLength, max({1}) x2FeatureLength, max({2}) x3FeatureLength
FROM inst
where {0} not like 'PrivacySuppressed' and {1} not like 'PrivacySuppressed' and {2} not like 'PrivacySuppressed'
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'])

In [ ]:
featureLengthDF = sqldf(maxQuery,locals())

In [ ]:
featureLengths = featureLengthDF.loc[0]

In [ ]:
x1FeatureLength = float(test['x1FeatureLength'])
x2FeatureLength = float(test['x2FeatureLength'])
x3FeatureLength = float(test['x3FeatureLength'])

part 2: applying length to normalize each feature

In [ ]:
normalizedQuery = """
SELECT 
{0}/{3} feature1normalized,
{1}/{4} feature2normalized,
{2}/{5} feature3normalized
FROM inst
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'], x1FeatureLength, x2FeatureLength, x3FeatureLength)

In [ ]:
normalizedDF = sqldf(normalizedQuery,locals())
normalizedDF 

multiplying preferences to feature

In [ ]:
prefVector = np.array([0.3,0.2,0.5])

In [ ]:
normalizedMatrix = normalizedDF.values

In [ ]:
normalizedMatrix

In [ ]:
ratingScore = np.dot(normalizedMatrix, prefVector)

In [ ]:
baseDF['Rating'] = ratingScore

In [ ]:
testQuery = """
SELECT count(*)
FROM baseDF
where rating is not NULL
"""

In [ ]:
output = sqldf(testQuery,locals())
output

apply filters

In [68]:
def filters_dict(filter1,filter2,filter3):
    filters_dict = {}
    filters_dict['filter1'] = filter1
    filters_dict['filter2'] = filter2
    filters_dict['filter3'] = filter3
    return filters_dict

In [69]:
filters_dict('OMENRUP_PELL_NFT_POOLED_SUPP', 'OMENRYP_PELL_FT_POOLED_SUPP', 'OMENRAP_PELL_FT_POOLED_SUPP')

{'filter1': 'OMENRUP_PELL_NFT_POOLED_SUPP',
 'filter2': 'OMENRYP_PELL_FT_POOLED_SUPP',
 'filter3': 'OMENRAP_PELL_FT_POOLED_SUPP'}